In [1]:
### FIRE Outflows CGM Interactions
### Astrophysics Project 2

In [2]:
# Import of Important Libraries
import sys, os, time, importlib, glob, pdb
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import OutflowsCGMInteractionTools.AnalysisTools as l
import OutflowsCGMInteractionTools.HalosData as HD
import OutflowsCGMInteractionTools.Config as c

In [3]:
# Print the Config File
print("Stampede Simulations")
print("The simulations names in the config file are:",c.ListOfStampedeSimulationsNames)
print("The simulations key names in the config file are:",c.ListOfStampedekeysOfTheSimulationsNames)
print("The snapshot numbers in the config file are:",c.ListOfStampedeSnapshotsNumbers)
print("")

# Creates the List of The Simulationes from the Config File and the Relevant Paths for Them
ListOfStampedeSimulationsNames=c.ListOfStampedeSimulationsNames
ListOfStampedeSimulationsPaths=[]
for i in ListOfStampedeSimulationsNames:
    SnapDirect=c.StampedeSnapDirect
    SnapDirect=SnapDirect+i
    ListOfStampedeSimulationsPaths.append(SnapDirect)
print("The simulations paths for Stampede are: ",ListOfStampedeSimulationsPaths)
print("The snapshots numbers for Stampede are: ",c.ListOfStampedeSnapshotsNumbers)
print("")

# This Block Will Create the Full Paths Directories for the Chosen Simulations and Sanpshot Numbers
ListOfSnapshotsNumbersForStampede=c.ListOfStampedeSnapshotsNumbers
ListOfOutputFolderPathsForStampede=[]
for i in ListOfStampedeSimulationsPaths:
    temp=i+"/output"
    ListOfOutputFolderPathsForStampede.append(temp)
ListOfFullPathsForStampede=[]
for i in ListOfOutputFolderPathsForStampede:
    OutputFiles=os.listdir(i)
    for j in ListOfSnapshotsNumbersForStampede:
        option1="snapshot_"+j+".hdf5"
        option2="snapdir_"+j
        if(option1 in OutputFiles):
            FullPath=i+"/"+option1
        if(option2 in OutputFiles):
            FullPath=i+"/"+option2
        ListOfFullPathsForStampede.append(FullPath) 
print("The full paths for Stampede simulations and snapshots are:")
print(ListOfFullPathsForStampede)

Stampede Simulations
The simulations names in the config file are: ['m12i_res7100']
The simulations key names in the config file are: ['m12i']
The snapshot numbers in the config file are: ['465', '464']

The simulations paths for Stampede are:  ['/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_res7100']
The snapshots numbers for Stampede are:  ['465', '464']

The full paths for Stampede simulations and snapshots are:
['/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_res7100/output/snapdir_465', '/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_res7100/output/snapdir_464']


In [4]:
def FindTheClosestStarToTheCenter(ListOfRsInThisLimit):
    IndexOfTheStarThatIsClosestTheCenter=0
    ClosestR=ListOfRsInThisLimit[0]
    for i in range(0,len(ListOfRsInThisLimit)):
        if(ListOfRsInThisLimit[i]<=ClosestR):
            ClosestR=ListOfRsInThisLimit[i]
            IndexOfTheStarThatIsClosestTheCenter=i
    return IndexOfTheStarThatIsClosestTheCenter

def ISM(Rvir,ListOfStarsMasses,ListOfCoordinates):# Calculates the ISM: First taking R_vir and multiply by 0.2, then sum all the masses within 
    # this distance, then multiply by 0.5 (by definition we need the half mass radius), then checks what radius contains this mass, then multiply 
    # this radius by 4 ---> this is the ISM radius
    ListOfCoordinates=np.array(ListOfCoordinates)
    ListOfRs=[]
    for i in range(0,len(ListOfCoordinates)):
        Coordinates=ListOfCoordinates[i]
        R=np.sqrt(Coordinates[0]**2+Coordinates[1]**2+Coordinates[2]**2)
        ListOfRs.append(R)
    RvirLimit=0.2*Rvir
    TotalMassInThisLimit=0
    ListOfStarsInThisLimit=[]
    ListOfRsInThisLimit=[]
    #print("Done1")
    for i in range(0,len(ListOfStarsMasses)):
        DistanceOfTheStar=ListOfRs[i]
        if(DistanceOfTheStar<RvirLimit):
            TotalMassInThisLimit=TotalMassInThisLimit+ListOfStarsMasses[i]
            ListOfStarsInThisLimit.append(ListOfStarsMasses[i])
            ListOfRsInThisLimit.append(ListOfRs[i])
    ListOfStarsInThisLimit=np.array(ListOfStarsInThisLimit)
    ListOfRsInThisLimit=np.array(ListOfRsInThisLimit)
    #print("Done2")
    HalfMassInThisLimit=0.5*TotalMassInThisLimit
    #print("Half mass value= ",HalfMassInThisLimit)
    FillTheHalfMassInThisLimit=0
    MaxRInThisLimit=RvirLimit
    CloseEnough=False
    while(CloseEnough==False):
        MaxRInThisLimit=MaxRInThisLimit/1.5
        FillTheHalfMassInThisLimit=0
        for i in range(0,len(ListOfStarsMasses)):
            DistanceOfTheStar=ListOfRs[i]
            if(DistanceOfTheStar<MaxRInThisLimit):
                FillTheHalfMassInThisLimit=FillTheHalfMassInThisLimit+ListOfStarsMasses[i]
        #print("Fill mass till reach the half mass value= ",FillTheHalfMassInThisLimit)
        #print("The difference is= HalfMassInThisLimit-FillTheHalfMassInThisLimit= ",HalfMassInThisLimit-FillTheHalfMassInThisLimit)
        prop=FillTheHalfMassInThisLimit/HalfMassInThisLimit
        #print("The propotion between them= FillTheHalfMassInThisLimit/HalfMassInThisLimit= ",prop)
        if(np.abs(prop-1)<0.1):
            CloseEnough=True
    return(4*MaxRInThisLimit)

def FindTheIndexOfParticleFromIDsArray(ParticlesIDsArray,ParticleID):
    index=np.where(ParticlesIDsArray == ParticleID)
    return index

In [5]:
# This Block Will Create Arrays With the Properties of the Gas Particles
AllRedshifts=[]# List of lists for all snapshots, each sublist is: [Redshift,Snapnum,IDS,Coordinates,R,Masses,Density,Temperature,NeutralH,SmoothingLengths]
AllRedshiftsICM=[]# List of lists for all snapshots only particles in the ICM area, each sublist is: [Redshift,Snapnum,IDS,Coordinates,R,Masses,Density,Temperature,NeutralH,SmoothingLengths]
AllParticelsIDs=np.array([])
ListOfSnapshotsParts=[]
forloopcount=0
for SnapDirect in ListOfFullPathsForStampede:
    ListOfSnapshotsParts=[]
    if(SnapDirect.endswith('.hdf5')):
        ListOfSnapshotsParts.append(SnapDirect)
    else:
        ListOfFiles=os.listdir(SnapDirect)
        for i in ListOfFiles:
            if (i.endswith('.hdf5')):
                ListOfSnapshotsParts.append(SnapDirect+'/'+i)# Append only hdf5 files to the snapshots parts list
    SimulationKey=c.ListOfStampedekeysOfTheSimulationsNames[forloopcount]
    Snap1=l.Snapshot(ListOfSnapshotsParts[0],[0,0,0],[0,0,0])# SnapName, not real center coordinates, not real center velocity
    z=Snap1.Redshift()
    print("The redshift of the given halo is:",z)
    snapnum=HD.Snapnum(SimulationKey,z)
    Mvir=HD.HaloMvir(SimulationKey,z)
    Rvir=HD.HaloRvir(SimulationKey,z)
    CenterCoordinates=HD.HaloCenterCoordinates(SimulationKey,z)
    CenterVelocity=HD.HaloCenterVelocity(SimulationKey,z)
    afactor=HD.HaloaFactor(SimulationKey,z)

    # This part intended to merge all the data from the different snapshot parts (if there are more than 1 part)
    StarsMasses=[]
    StarsCoordinates=[]
    TotalNumberOfParticles=0
    Masses=[]
    IDS=[]
    Coordinates=[]
    Temperature=[]
    GasR=[]
    Density=[]
    NeutralH=[]
    GasMasses=[]
    BoxesSizes=[]
    SmoothingLengths=[]
    
    for snapshotname in ListOfSnapshotsParts:
        Snap1=l.Snapshot(snapshotname,CenterCoordinates,CenterVelocity,a=afactor)# SnapName, real center coordinates, real center velocity
        
        NumberOfParticlesOfOneSnapshot=Snap1.number_of_particles()# 0-Gas,1-High Resolution Dark Matter,2-Dummy Particles,3-Dummy Particles,4-Stars,5-Black Holes
        TotalNumberOfParticles=TotalNumberOfParticles+NumberOfParticlesOfOneSnapshot[0]

        MassesOfOneSnapshotStars=Snap1.StarsMasses(4)# The masses of the stars
        StarsMasses.extend(MassesOfOneSnapshotStars)
        
        CoordinatesOfOneSnapShotStars=Snap1.coords(4)# The coordinates of the stars
        StarsCoordinates.extend(CoordinatesOfOneSnapShotStars)
        
        MassesOfOneSnapshotGasParticles=Snap1.masses(0)# The masses of the gas particles
        GasMasses.extend(MassesOfOneSnapshotGasParticles)
        
        IDSOfOneSnapshotGasParticles=Snap1.IDS(0)# The I.Ds of the gas particles
        IDS.extend(IDSOfOneSnapshotGasParticles)

        CoordinatesOfOneSnapShot=Snap1.coords(0)# The coordinates of the gas particles
        Coordinates.extend(CoordinatesOfOneSnapShot)

        TemperatureOfOneSnapShot=Snap1.Ts()# The temperature of the gas particles
        Temperature.extend(TemperatureOfOneSnapShot)

        GasROfOneSnapShot=Snap1.rs()# The r of the gas particles
        GasR.extend(GasROfOneSnapShot)

        DensityOfOneSnapShot=Snap1.rhos()# The density of the gas particles
        Density.extend(DensityOfOneSnapShot)

        NeutralHOfOneSnapshot=Snap1.neutralHydrogenAbundance(0)# Returns the neutral hydrogen fraction
        NeutralH.extend(NeutralHOfOneSnapshot)
        
        SmoothingLengthOfOneSnapshot=Snap1.SmoothingLength()
        SmoothingLengths.extend(SmoothingLengthOfOneSnapshot)
        
    ISM_Radius=ISM(Rvir,StarsMasses,StarsCoordinates)
    Coordinates=np.array(Coordinates)
    Temperature=np.array(Temperature)
    GasR=np.array(GasR)
    Density=np.array(Density)
    NeutralH=np.array(NeutralH)
    GasMasses=np.array(GasMasses)
    IDS=np.array(IDS)
    SmoothingLengths=np.array(SmoothingLengths)
    TotalNumberOfGasParticlesInICM=0
    
    # Now we want to select only particles that are in the ISM area and build their properties arays
    ICM_Gas_IDS=[]
    ICM_Gas_Coordinates=[]
    ICM_Gas_R=[]
    ICM_Gas_Masses=[]
    ICM_Gas_Density=[]
    ICM_Gas_Temperature=[]
    ICM_Gas_NeutralH=[]
    ICM_Gas_SmoothingLengths=[]   
    for i in range(0,len(GasR)):
        if(GasR[i]<=ISM_Radius):
            ICM_Gas_IDS.append(IDS[i])
            ICM_Gas_Coordinates.append(Coordinates[i])
            ICM_Gas_R.append(GasR[i])
            ICM_Gas_Masses.append(GasMasses[i])
            ICM_Gas_Density.append(Density[i])
            ICM_Gas_Temperature.append(Temperature[i])
            ICM_Gas_NeutralH.append(NeutralH[i])
            ICM_Gas_SmoothingLengths.append(SmoothingLengths[i])
            TotalNumberOfGasParticlesInICM=TotalNumberOfGasParticlesInICM+1
    
    ICM_Gas_IDS=np.array(ICM_Gas_IDS)
    ICM_Gas_Coordinates=np.array(ICM_Gas_Coordinates)
    ICM_Gas_R=np.array(ICM_Gas_R)
    ICM_Gas_Masses=np.array(ICM_Gas_Masses)
    ICM_Gas_Density=np.array(ICM_Gas_Density)
    ICM_Gas_Temperature=np.array(ICM_Gas_Temperature)
    ICM_Gas_NeutralH=np.array(ICM_Gas_NeutralH)
    ICM_Gas_SmoothingLengths=np.array(ICM_Gas_SmoothingLengths)
    
    print("ISM radius is: ",ISM_Radius)
    print("The total number of gas particles is:",TotalNumberOfParticles)
    print("The total number of gas particles in the ISM is:",TotalNumberOfGasParticlesInICM)
    tempRedshift=[z,snapnum,IDS,Coordinates,GasR,GasMasses,Density,Temperature,NeutralH,SmoothingLengths]
    tempRedshiftICM=[z,snapnum,ICM_Gas_IDS,ICM_Gas_Coordinates,ICM_Gas_R,ICM_Gas_Masses,ICM_Gas_Density,ICM_Gas_Temperature,ICM_Gas_NeutralH,ICM_Gas_SmoothingLengths]
    AllRedshifts.append(tempRedshift)
    AllRedshiftsICM.append(tempRedshiftICM)
    print("#############################################################################################################")
AllRedshifts=np.array(AllRedshifts)
AllRedshiftsICM=np.array(AllRedshiftsICM)

The redshift of the given halo is: 0.25099595522592044
M_vir of the given halo is: 6.80e+11
R_vir of the given halo is: 158.16997582879168
The closet redshift to the given redshift is: 0.25099599
The center coordinates of the given halo is: c= [29450.90216764, 30798.5765683, 32256.40448552]
The velocity of the center of the given halo is: Vc= [-49.73, 75.31, 96.97]
a factor of the given halo is: 0.799363073897623
loaded PartType4, Masses
loaded PartType4, Coordinates
loaded PartType0, Masses
loaded PartType0, ParticleIDs
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
loaded PartType0, Density
loaded PartType0, NeutralHydrogenAbundance
loaded PartType0, SmoothingLength
loaded PartType4, Masses
loaded PartType4, Coordinates
loaded PartType0, Masses
loaded PartType0, ParticleIDs
loaded PartType0, Coordinates
loaded PartType0, InternalEnergy
loaded PartType0, Density
loaded PartType0, NeutralHydrogenAbundance
loaded PartType0, SmoothingLength
loaded PartType4, Masses
loaded

/tmp/ipykernel_200779/1404489631.py:127: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  AllRedshifts=np.array(AllRedshifts)
/tmp/ipykernel_200779/1404489631.py:128: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  AllRedshiftsICM=np.array(AllRedshiftsICM)


In [6]:
# Track the particles at the ISM area from redshift 0.25 to 0.5 = snapshot 465 to 382
ParticlesRsAmongRedshifts=[]# Array of arrays each element is: [Particle ID, R_465, R_464, R_463,...,R_382]
GasParticlesIDsOfRedshift0_25ISM=AllRedshiftsICM[0][2]# Array of all particles in ICM area for redshift 0.25
for i in range(0,100):# Taking number of particles from this list, the ideal is whole particles in GasParticlesIDsOfRedshift0_25ISM
    TempRTrack=np.array([GasParticlesIDsOfRedshift0_25ISM[i]])# First element in the subarray is the particle ID
    for j in range(0,len(AllRedshifts)):# For each of the snapshots 465 to 382
        TheIndexOfTheParticle=FindTheIndexOfParticleFromIDsArray(AllRedshifts[j][2],GasParticlesIDsOfRedshift0_25ISM[i])# Find the index of this particle in a specific snapshot
        if(len(AllRedshifts[j][4][TheIndexOfTheParticle])==1):# Check if R of this particle is ok
            TempRTrack=np.append(TempRTrack,AllRedshifts[j][4][TheIndexOfTheParticle])# Append the relvant R of the particle of specific snapshot to the subarray 
        else:
            TempRTrack=np.append(TempRTrack,None)
    ParticlesRsAmongRedshifts.append(TempRTrack)# Finished taking all R for the snapshots for a specific particle and append the subarray to the main array 

print("The number of tracked particles in the ISM from z=0.25 to z=0.5 is: ",len(ParticlesRsAmongRedshifts))
print("Tracking data:")
print("[Particle I.D , R_465 , R_464 , ... ]")
for SpecificParticle in ParticlesRsAmongRedshifts:
    print(SpecificParticle)
print("#############################################################################################################")

The number of tracked particles in the ISM from z=0.25 to z=0.5 is:  100
Tracking data:
[Particle I.D , R_465 , R_464 , ... ]
[1.26108787e+07 1.10971303e+01 1.03755415e+01]
[1.58638291e+07 1.10924789e+01 1.10727095e+01]
[1.02180778e+07 1.10080201e+01 1.07913045e+01]
[1.03253458e+07 1.10318385e+01 1.09801961e+01]
[1.46655621e+07 1.10401875e+01 1.03745989e+01]
[2.89286760e+07 1.10321890e+01 1.10344549e+01]
[1.86701781e+07 1.10663120e+01 1.08747933e+01]
[9.67965600e+06 1.10525196e+01 1.09403879e+01]
[2.47212413e+07 1.10250706e+01 1.11550707e+01]
[1.31525660e+07 1.10845938e+01 1.03194579e+01]
[1.28246902e+07 1.10592393e+01 1.08807976e+01]
[9.78434940e+06 1.10404676e+01 1.06731206e+01]
[1.31647229e+07 1.10469751e+01 1.09943241e+01]
[2.78623261e+07 1.10537019e+01 1.09395982e+01]
[1.37262951e+07 1.10294703e+01 1.07957195e+01]
[1.35074926e+07 1.10674975e+01 1.09643938e+01]
[2.63556671e+07 1.10978666e+01 1.11605055e+01]
[2.01608155e+07 1.10873375e+01 1.08256744e+01]
[3.71958062e+07 1.10365767e+